In [1]:
#load needed imports
import pandas as pd
import datetime
import networkx as nx
import sys
import time as tm
import numpy as np

In [2]:
#prepare functions and constants

DAY_SECONDS = 86400
HOUR_SECONDS = 3600 
MINUTE_SECONDS = 60



#functions below are needed to deal with time format provided in exercise's data
def fix_time(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    hours = hours % 24
    return f'{hours:02d}:{minutes:02d}:{seconds:02d}'

def time_to_seconds(time):
    return time.second + time.minute*60 + time.hour*3600

def seconds_to_hour_and_minute(seconds):
	return "hours: ", seconds // HOUR_SECONDS,"minutes: ", (seconds % HOUR_SECONDS) // MINUTE_SECONDS

In [3]:
#import data and first look
df = pd.read_csv("connection_graph.csv", low_memory=False)
df.head()

,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


In [4]:
#clean data
df.rename(columns={df.columns[0]: 'del'}, inplace=True)

df.drop(columns=['del','company'], inplace=True)
df.drop_duplicates(inplace=True)
df = df[df['start_stop'] != df['end_stop']]

#fix time format
df['departure_time'] = df['departure_time'].apply(fix_time)
df['arrival_time'] = df['arrival_time'].apply(fix_time)
df['departure_time'] = pd.to_datetime(df['departure_time'], format='%H:%M:%S').dt.time
df['arrival_time'] = pd.to_datetime(df['arrival_time'], format='%H:%M:%S').dt.time

#add time interpretation in seconds
df['departure_time_seconds'] = df['departure_time'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)
df['arrival_time_seconds'] = df['arrival_time'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)

df['id'] = range(0, len(df))
df.set_index('id', inplace=True)

df.head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_seconds,arrival_time_seconds
id,,,,,,,,,,,
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540


In [5]:
#now we need to prepare the list of stops we are working with, dropping the duplicates to provide data integrity
stops = df[['start_stop', 'start_stop_lat', 'start_stop_lon']].drop_duplicates(subset=['start_stop'], keep='first').sort_values(by='start_stop')

#no longer need for lat and lon info after getting stops so we need to get rid of them in main data

df.drop(columns=['start_stop_lat','start_stop_lon', 'end_stop_lat','end_stop_lon'], inplace=True)

print(len(stops))
stops.head()
df.head()



938


,line,departure_time,arrival_time,start_stop,end_stop,departure_time_seconds,arrival_time_seconds
id,,,,,,,
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,75420,75540


In [6]:
#graph creation of stops
stop_graph = nx.DiGraph()

def create_stop_nodes(conn):
    stop_graph.add_node(conn['start_stop'], lat=conn['start_stop_lat'], lon=conn['start_stop_lon'])

stops.apply(create_stop_nodes, axis=1)

print(len(stop_graph.nodes))
print(list(stop_graph.nodes.data())[None:10:None])

938
[('8 Maja', {'lat': 51.11366919, 'lon': 17.09111971}), ('AUCHAN', {'lat': 51.0520648, 'lon': 16.97405354}), ('Adamczewskich', {'lat': 51.12084396, 'lon': 16.86953486}), ('Adamieckiego', {'lat': 51.07425917, 'lon': 16.96299503}), ('Aleja Architektów', {'lat': 51.13881871, 'lon': 16.92905866}), ('Aleja Pracy', {'lat': 51.0918696, 'lon': 16.98537334}), ('Aleja Wędrowców', {'lat': 51.0879861, 'lon': 16.99188243}), ('Arabska', {'lat': 51.05115765, 'lon': 17.0923256}), ('Arachidowa', {'lat': 51.15417321, 'lon': 16.89901728}), ('Arena', {'lat': 51.094134, 'lon': 17.026404})]


In [7]:
#adding edges
def create_edges(conn):
    if not stop_graph.has_edge(conn['start_stop'], conn['end_stop']):
        stop_graph.add_edge(conn['start_stop'], conn['end_stop'], schedule=[])

    # Append the new connection data to the schedule list
    stop_graph[conn['start_stop']][conn['end_stop']]['schedule'].append(conn.to_dict())

df.apply(create_edges, axis=1)

print(stop_graph['Ostrowskiego']['FAT']['schedule'][0])

{'line': '107', 'departure_time': datetime.time(7, 11), 'arrival_time': datetime.time(7, 12), 'start_stop': 'Ostrowskiego', 'end_stop': 'FAT', 'departure_time_seconds': 25860, 'arrival_time_seconds': 25920}


In [8]:
# Function to retrieve path
def return_path_to_destination(start, end, line, best_routes_df):
    path = []
    (line_info, when, stop) = (f"leave line: { line[end]['line']}", line[end]['arrival_time'], f"at: {end}")
    path.append((line_info, when, stop))
    end = best_routes_df.loc['previous_stop', end]
    current_line = line[end]['line']
    while end != start:
        previous_stop_info = line[best_routes_df.loc['previous_stop', end]]

        if previous_stop_info is not None and previous_stop_info['line'] != current_line:
            (line_info, when, stop) = (f"enter line: { line[end]['line']}", line[end]['departure_time'], f"at: {line[end]['start_stop']}")
            path.append((line_info, when, stop))

            tmp_stop = best_routes_df.loc['previous_stop', end]
            (line_info, when, stop) = (f"leave line: { line[tmp_stop]['line']}", line[tmp_stop]['arrival_time'], f"at: {tmp_stop}")
            path.append((line_info, when, stop))

        future_end = best_routes_df.loc['previous_stop', end]

        if future_end != start:
            end = future_end
            current_line = line[end]['line']
        else:
            (line_info, when, stop) = (f"enter line: { line[end]['line']}", line[end]['departure_time'], f"at: {line[end]['start_stop']}")
            path.append((line_info, when, stop))
            end = future_end
    return path

def print_path(path):
    for i in range(len(path) - 1, -1, -1):
        print(path[i])

# Cost of travel's time
def count_total_cost(arrival_time_seconds, start_time_seconds):
    return (arrival_time_seconds - start_time_seconds + DAY_SECONDS) % DAY_SECONDS

def min_cost(unchecked_stops, best_routes_df):
    min_cost = best_routes_df.loc['min_cost', unchecked_stops[0]]
    min_stop = unchecked_stops[0]

    for stop in unchecked_stops:
        if best_routes_df.loc['min_cost', stop] < min_cost:
            min_cost = best_routes_df.loc['min_cost', stop]
            min_stop = stop
    return min_stop

def best_connection(schedule, current_time_seconds, current_cost):
    best_con = None
    best_cost = float('inf')

    for connection in schedule:
        waiting_time = (connection['departure_time_seconds'] - current_time_seconds + DAY_SECONDS) % DAY_SECONDS
        travel_time = ((connection['arrival_time_seconds'] - connection['departure_time_seconds'] + DAY_SECONDS) % DAY_SECONDS)
        total_cost = current_cost + waiting_time + travel_time

        if best_cost > total_cost:
            best_cost = total_cost
            best_con = connection

    return (best_con, best_cost)


In [9]:
import time as tm 

def dijkstra(graph, start, time):
    algorithm_start_time = tm.time()  # Record the start time of the algorithm
    
    # Dictionary with the best connections for each stop
    line = {}
    for node in graph.nodes:
        line[node] = None

    # DataFrame with unique stops as columns, which will contain the outcome
    best_routes_df = pd.DataFrame(columns=graph.nodes)
    best_routes_df.loc['min_cost'] = np.inf
    best_routes_df.loc['previous_stop'] = ""
    best_routes_df.loc['arrival_time_seconds'] = None

    # Setting initial values
    current_time_seconds = time_to_seconds(time)
    best_routes_df.loc['min_cost', start] = 0
    best_routes_df.loc['best_arrival_time_seconds', start] = current_time_seconds

    checked_stops = []
    unchecked_stops = list(graph.nodes)
    
    while unchecked_stops:
        # Find stop with minimum cost, if it's not in unchecked_stops
        current_stop = min_cost(unchecked_stops, best_routes_df)

        unchecked_stops.remove(current_stop)
        checked_stops.append(current_stop)

        current_time_seconds = best_routes_df.loc['best_arrival_time_seconds', current_stop]
        current_cost = best_routes_df.loc['min_cost', current_stop]

        for neighbor in graph.neighbors(current_stop):
            (connection, total_cost) = best_connection(graph[current_stop][neighbor]['schedule'], current_time_seconds, current_cost)
            if best_routes_df.loc['min_cost', neighbor] > total_cost:
                best_routes_df.loc['min_cost', neighbor] = total_cost
                best_routes_df.loc['previous_stop', neighbor] = current_stop
                best_routes_df.loc['best_arrival_time_seconds', neighbor] = connection['arrival_time_seconds']
                line[neighbor] = connection

    print("Route found.")
    algorithm_duration = tm.time() - algorithm_start_time  # Calculate the algorithm's duration
    return (line, best_routes_df, algorithm_duration)


In [10]:
import sys

start = "Babimojska"
stop = "Biegasa"
start_time = datetime.time(16,58, 0)

(line, best_routes_df,algorithm_duration) = dijkstra(stop_graph, start, start_time)

path = return_path_to_destination(start, stop, line, best_routes_df)
print_path(path)
total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Route found.
('enter line: 149', datetime.time(16, 59), 'at: Babimojska')
('leave line: 149', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('enter line: 19', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('leave line: 19', datetime.time(17, 30), 'at: ZOO')
('enter line: 2', datetime.time(17, 31), 'at: ZOO')
('leave line: 2', datetime.time(17, 33), 'at: Tramwajowa')
('enter line: 4', datetime.time(17, 33), 'at: Tramwajowa')
('leave line: 143', datetime.time(17, 38), 'at: Biegasa')


Koszt  ('hours: ', 0, 'minutes: ', 40)
Czas trwania  6.032975435256958


In [11]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 23, 0)

(line, best_routes_df,algorithm_duration) = dijkstra(stop_graph, start, start_time)

path = return_path_to_destination(start, stop, line, best_routes_df)
print_path(path)
total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Route found.
('enter line: 106', datetime.time(10, 29), 'at: PORT LOTNICZY')
('leave line: 106', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('enter line: 122', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('leave line: 122', datetime.time(10, 46), 'at: Strzegomska (krzyżówka)')
('enter line: 23', datetime.time(10, 47), 'at: Strzegomska (krzyżówka)')
('leave line: 23', datetime.time(11, 3), 'at: Arkady (Capitol)')
('enter line: D', datetime.time(11, 5), 'at: Arkady (Capitol)')
('leave line: D', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('enter line: 1', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('leave line: 1', datetime.time(11, 18), 'at: Kliniki - Politechnika Wrocławska')
('enter line: 115', datetime.time(11, 22), 'at: Kliniki - Politechnika Wrocławska')
('leave line: 115', datetime.time(11, 33), 'at: Swojczyce')


Koszt  ('hours: ', 1, 'minutes: ', 10)
Czas trwania  6.150092363357544


In [12]:
# Algorytm A*
from math import sqrt

# Constants to estimate approximate time to destination
average_speed_mpk_km_s = 16.6 / 3600  # average velocity in Wrocław [km/s]
degrees_to_km = 111 

def best_connection_min_time(schedule, current_time_seconds, current_cost, previous_connection=None):
    best_connection = None
    best_cost = float('inf')

    for connection in schedule:
        waiting_time = (connection['departure_time_seconds'] - current_time_seconds + DAY_SECONDS) % DAY_SECONDS
        travel_time = (connection['arrival_time_seconds'] - connection['departure_time_seconds'] + DAY_SECONDS) % DAY_SECONDS
        total_cost = current_cost + waiting_time + travel_time

        if total_cost < best_cost:
            best_cost = total_cost
            best_connection = connection

    return (best_connection, best_cost)

def best_connection_min_transfers(schedule, current_time_seconds, current_cost, previous_connection):
    previous_line = ""
    if previous_connection:
        previous_line = previous_connection['line']
    best_connection = None
    best_cost = float('inf')

    for connection in schedule:
        waiting_time = (connection['departure_time_seconds'] - current_time_seconds + 86400) % 86400
        travel_time = ((connection['arrival_time_seconds'] - connection['departure_time_seconds'] + 86400) % 86400)
        total_cost = current_cost + waiting_time + travel_time

        if previous_line and connection['line'] != previous_line:
            total_cost += 1000  # Penalizing for a line change

        if best_cost > total_cost:
            best_cost = total_cost
            best_connection = connection

    return (best_connection, best_cost)

def h_manhattan(current_node, goal_node):
    # Calculating Manhattan distance based on latitude and longitude
    distance_km = degrees_to_km * sqrt((current_node['lat'] - goal_node['lat'])**2 + (current_node['lon'] - goal_node['lon'])**2)
    return distance_km / average_speed_mpk_km_s

def h_euclidean(current_node, goal_node):
    # Calculating Euclidean distance for an approximation
    distance_km = degrees_to_km * (abs(current_node['lat'] - goal_node['lat']) + abs(current_node['lon'] - goal_node['lon']))
    return distance_km / average_speed_mpk_km_s

def haversine(current_node, goal_node):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [current_node['lat'], current_node['lon'], goal_node['lat'], goal_node['lon']])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance / average_speed_mpk_km_s


def astar(graph, start, end, time, h=h_euclidean, best_connection_function=best_connection_min_transfers):
    algorithm_start_time = tm.time()

    # Initialize all nodes in the graph with default g, h, and f values
    for node in graph.nodes():
        graph.nodes[node]['g'] = 0  # Cost from start to node
        graph.nodes[node]['h'] = 0  # Heuristic estimate from node to end
        graph.nodes[node]['f'] = 0  # Total cost (g + h)

    # Creating DataFrame with unique stops as columns to store the outcomes
    best_routes_df = pd.DataFrame(columns=graph.nodes)
    best_routes_df.loc['min_cost'] = float('inf')  # Initialize min_cost for all nodes
    best_routes_df.loc['previous_stop'] = ""  # To track the best path
    best_routes_df.loc['arrival_time_seconds'] = None  # Store arrival times for best paths

    # Setting initial values
    current_time_seconds = time_to_seconds(time)
    best_routes_df.loc['min_cost', start] = 0  # Start node has a cost of 0 to reach
    best_routes_df.loc['best_arrival_time_seconds', start] = current_time_seconds

    # Lists to track which stops have been checked or need to be checked
    checked_stops = []  # Closed set
    unchecked_stops = list(graph.nodes)  # Open set, initially contains all nodes

    # Creating a dictionary to store the best connection for each stop
    line = {node: None for node in graph.nodes}

    while unchecked_stops:
        current_stop = None
        lowest_cost = float('inf')
        
        # Find the stop with the lowest f value (min_cost) among the unchecked stops
        current_stop = min_cost(unchecked_stops, best_routes_df)
        lowest_cost = best_routes_df.loc['min_cost', current_stop]
        current_time_seconds = best_routes_df.loc['best_arrival_time_seconds', current_stop]

        # Break if the end stop is reached
        if current_stop == end:
            break
        
        # Remove the current stop from unchecked_stops and add it to checked_stops
        unchecked_stops.remove(current_stop)
        checked_stops.append(current_stop)

        # Explore all neighbors of the current stop
        for neighbor in graph.neighbors(current_stop):
            connection, cost = best_connection_function(graph[current_stop][neighbor]['schedule'], current_time_seconds, lowest_cost, line[current_stop])

            # If a better path to the neighbor is found, update the best routes dataframe
            if best_routes_df.loc['min_cost', neighbor] > cost:
                best_routes_df.loc['min_cost', neighbor] = cost
                best_routes_df.loc['previous_stop', neighbor] = current_stop
                best_routes_df.loc['best_arrival_time_seconds', neighbor] = connection['arrival_time_seconds']
                line[neighbor] = connection

            # Update heuristic and total cost values for neighbors
            if neighbor not in unchecked_stops and neighbor not in checked_stops:
                unchecked_stops.append(neighbor)
                graph.nodes[neighbor]['h'] = h(graph.nodes[neighbor], graph.nodes[end])
                graph.nodes[neighbor]['g'] = graph.nodes[current_stop]['g'] + cost
                graph.nodes[neighbor]['f'] = graph.nodes[neighbor]['g'] + graph.nodes[neighbor]['h']
            elif graph.nodes[neighbor]['g'] > graph.nodes[current_stop]['g'] + cost:
                graph.nodes[neighbor]['g'] = graph.nodes[current_stop]['g'] + cost
                graph.nodes[neighbor]['f'] = graph.nodes[neighbor]['g'] + graph.nodes[neighbor]['h']
                if neighbor in checked_stops:
                    unchecked_stops.append(neighbor)
                    checked_stops.remove(neighbor)

    # Calculate the total duration of the algorithm execution
    algorithm_duration = tm.time() - algorithm_start_time
    return (line, best_routes_df, algorithm_duration)


In [13]:
#A-star min time examples haversine

In [14]:
start = "Babimojska"
stop = "Biegasa"
start_time = datetime.time(16, 58, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=haversine,best_connection_function=best_connection_min_time)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 149', datetime.time(16, 59), 'at: Babimojska')
('leave line: 149', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('enter line: 19', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('leave line: 19', datetime.time(17, 30), 'at: ZOO')
('enter line: 2', datetime.time(17, 31), 'at: ZOO')
('leave line: 2', datetime.time(17, 33), 'at: Tramwajowa')
('enter line: 4', datetime.time(17, 33), 'at: Tramwajowa')
('leave line: 143', datetime.time(17, 38), 'at: Biegasa')


Koszt  ('hours: ', 0, 'minutes: ', 40)
Czas trwania  4.809890985488892


In [15]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 23, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=haversine,best_connection_function=best_connection_min_time)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 106', datetime.time(10, 29), 'at: PORT LOTNICZY')
('leave line: 106', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('enter line: 122', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('leave line: 122', datetime.time(10, 46), 'at: Strzegomska (krzyżówka)')
('enter line: 23', datetime.time(10, 47), 'at: Strzegomska (krzyżówka)')
('leave line: 23', datetime.time(11, 3), 'at: Arkady (Capitol)')
('enter line: D', datetime.time(11, 5), 'at: Arkady (Capitol)')
('leave line: D', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('enter line: 1', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('leave line: 1', datetime.time(11, 18), 'at: Kliniki - Politechnika Wrocławska')
('enter line: 115', datetime.time(11, 22), 'at: Kliniki - Politechnika Wrocławska')
('leave line: 115', datetime.time(11, 33), 'at: Swojczyce')


Koszt  ('hours: ', 1, 'minutes: ', 10)
Czas trwania  5.419087648391724


In [16]:
#A-start min transfer examples haversine

In [17]:
start = "Babimojska"
stop = "Biegasa"
start_time = datetime.time(16, 58, 0)


(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=haversine,best_connection_function=best_connection_min_transfers)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 149', datetime.time(16, 59), 'at: Babimojska')
('leave line: 149', datetime.time(17, 15), 'at: Dworzec Główny (Dworcowa)')
('enter line: 114', datetime.time(17, 19), 'at: Dworzec Główny (Dworcowa)')
('leave line: 114', datetime.time(17, 28), 'at: Armii Krajowej')
('enter line: 143', datetime.time(17, 35), 'at: Armii Krajowej')
('leave line: 143', datetime.time(17, 39), 'at: Biegasa')


Koszt  ('hours: ', 0, 'minutes: ', 41)
Czas trwania  4.071877717971802


In [18]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 23, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=haversine,best_connection_function=best_connection_min_transfers)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 106', datetime.time(10, 29), 'at: PORT LOTNICZY')
('leave line: 106', datetime.time(11, 3), 'at: Renoma')
('enter line: 17', datetime.time(11, 5), 'at: Renoma')
('leave line: 17', datetime.time(11, 30), 'at: SĘPOLNO')
('enter line: 115', datetime.time(11, 33), 'at: SĘPOLNO')
('leave line: 115', datetime.time(11, 37), 'at: Swojczyce')


Koszt  ('hours: ', 1, 'minutes: ', 14)
Czas trwania  5.234850168228149


In [19]:
#A-star min time examples euklides

In [20]:
start = "Babimojska"
stop = "Biegasa"
start_time = datetime.time(16, 58, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_euclidean,best_connection_function=best_connection_min_time)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 149', datetime.time(16, 59), 'at: Babimojska')
('leave line: 149', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('enter line: 19', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('leave line: 19', datetime.time(17, 30), 'at: ZOO')
('enter line: 2', datetime.time(17, 31), 'at: ZOO')
('leave line: 2', datetime.time(17, 33), 'at: Tramwajowa')
('enter line: 4', datetime.time(17, 33), 'at: Tramwajowa')
('leave line: 143', datetime.time(17, 38), 'at: Biegasa')


Koszt  ('hours: ', 0, 'minutes: ', 40)
Czas trwania  4.924313068389893


In [21]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 23, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_euclidean,best_connection_function=best_connection_min_time)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 106', datetime.time(10, 29), 'at: PORT LOTNICZY')
('leave line: 106', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('enter line: 122', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('leave line: 122', datetime.time(10, 46), 'at: Strzegomska (krzyżówka)')
('enter line: 23', datetime.time(10, 47), 'at: Strzegomska (krzyżówka)')
('leave line: 23', datetime.time(11, 3), 'at: Arkady (Capitol)')
('enter line: D', datetime.time(11, 5), 'at: Arkady (Capitol)')
('leave line: D', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('enter line: 1', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('leave line: 1', datetime.time(11, 18), 'at: Kliniki - Politechnika Wrocławska')
('enter line: 115', datetime.time(11, 22), 'at: Kliniki - Politechnika Wrocławska')
('leave line: 115', datetime.time(11, 33), 'at: Swojczyce')


Koszt  ('hours: ', 1, 'minutes: ', 10)
Czas trwania  5.516069650650024


In [22]:
#A start min transfers euklides

In [23]:
start = "Babimojska"
stop = "Biegasa"
start_time = datetime.time(16, 58, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_euclidean,best_connection_function=best_connection_min_transfers)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 149', datetime.time(16, 59), 'at: Babimojska')
('leave line: 149', datetime.time(17, 15), 'at: Dworzec Główny (Dworcowa)')
('enter line: 114', datetime.time(17, 19), 'at: Dworzec Główny (Dworcowa)')
('leave line: 114', datetime.time(17, 28), 'at: Armii Krajowej')
('enter line: 143', datetime.time(17, 35), 'at: Armii Krajowej')
('leave line: 143', datetime.time(17, 39), 'at: Biegasa')


Koszt  ('hours: ', 0, 'minutes: ', 41)
Czas trwania  4.126176357269287


In [24]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 23, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_euclidean,best_connection_function=best_connection_min_transfers)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 106', datetime.time(10, 29), 'at: PORT LOTNICZY')
('leave line: 106', datetime.time(11, 3), 'at: Renoma')
('enter line: 17', datetime.time(11, 5), 'at: Renoma')
('leave line: 17', datetime.time(11, 30), 'at: SĘPOLNO')
('enter line: 115', datetime.time(11, 33), 'at: SĘPOLNO')
('leave line: 115', datetime.time(11, 37), 'at: Swojczyce')


Koszt  ('hours: ', 1, 'minutes: ', 14)
Czas trwania  5.233227491378784


In [25]:
#A start min time manhattan

In [26]:
start = "Babimojska"
stop = "Biegasa"
start_time = datetime.time(16, 58, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_manhattan,best_connection_function=best_connection_min_time)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 149', datetime.time(16, 59), 'at: Babimojska')
('leave line: 149', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('enter line: 19', datetime.time(17, 25), 'at: PL. GRUNWALDZKI')
('leave line: 19', datetime.time(17, 30), 'at: ZOO')
('enter line: 2', datetime.time(17, 31), 'at: ZOO')
('leave line: 2', datetime.time(17, 33), 'at: Tramwajowa')
('enter line: 4', datetime.time(17, 33), 'at: Tramwajowa')
('leave line: 143', datetime.time(17, 38), 'at: Biegasa')


Koszt  ('hours: ', 0, 'minutes: ', 40)
Czas trwania  4.78825306892395


In [27]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 23, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_manhattan,best_connection_function=best_connection_min_time)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 106', datetime.time(10, 29), 'at: PORT LOTNICZY')
('leave line: 106', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('enter line: 122', datetime.time(10, 42), 'at: MIŃSKA (Rondo Rotm. Pileckiego)')
('leave line: 122', datetime.time(10, 46), 'at: Strzegomska (krzyżówka)')
('enter line: 23', datetime.time(10, 47), 'at: Strzegomska (krzyżówka)')
('leave line: 23', datetime.time(11, 3), 'at: Arkady (Capitol)')
('enter line: D', datetime.time(11, 5), 'at: Arkady (Capitol)')
('leave line: D', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('enter line: 1', datetime.time(11, 16), 'at: PL. GRUNWALDZKI')
('leave line: 1', datetime.time(11, 18), 'at: Kliniki - Politechnika Wrocławska')
('enter line: 115', datetime.time(11, 22), 'at: Kliniki - Politechnika Wrocławska')
('leave line: 115', datetime.time(11, 33), 'at: Swojczyce')


Koszt  ('hours: ', 1, 'minutes: ', 10)
Czas trwania  5.456471920013428


In [28]:
#A start min transfers manhattan

In [29]:
start = "Babimojska"
stop = "Biegasa"
start_time = datetime.time(16, 58, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_manhattan,best_connection_function=best_connection_min_transfers)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 149', datetime.time(16, 59), 'at: Babimojska')
('leave line: 149', datetime.time(17, 15), 'at: Dworzec Główny (Dworcowa)')
('enter line: 114', datetime.time(17, 19), 'at: Dworzec Główny (Dworcowa)')
('leave line: 114', datetime.time(17, 28), 'at: Armii Krajowej')
('enter line: 143', datetime.time(17, 35), 'at: Armii Krajowej')
('leave line: 143', datetime.time(17, 39), 'at: Biegasa')


Koszt  ('hours: ', 0, 'minutes: ', 41)
Czas trwania  4.224871873855591


In [30]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 23, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(stop_graph, start, stop, start_time, h=h_manhattan,best_connection_function=best_connection_min_transfers)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('enter line: 106', datetime.time(10, 29), 'at: PORT LOTNICZY')
('leave line: 106', datetime.time(11, 3), 'at: Renoma')
('enter line: 17', datetime.time(11, 5), 'at: Renoma')
('leave line: 17', datetime.time(11, 30), 'at: SĘPOLNO')
('enter line: 115', datetime.time(11, 33), 'at: SĘPOLNO')
('leave line: 115', datetime.time(11, 37), 'at: Swojczyce')


Koszt  ('hours: ', 1, 'minutes: ', 14)
Czas trwania  5.388409614562988
